# Interpolate all of the data to H3 grid

This notebook interpolates both variables we aim to model (air quality and house price) to the H3 level 9 grid. At the same time, it interpolates all of the explanatory variables to the same grid. 

The resulting H3 grid, covering England, will serve as a basis for nation-wide predictive models and for any future deployment of the Demoland App.

In [1]:
import gc
from time import time

import pandas as pd
import geopandas as gpd
import xarray as xr
import tobler
import numpy as np
from itertools import product
import osmnx as ox
import dask_geopandas

from tobler.util import h3fy

In [2]:
data_folder = "../../../demoland_data"

## Create H3 grid

In [3]:
gb = gpd.read_file(f"{data_folder}/raw/geoBoundaries-GBR-ADM1-all.zip")

In [9]:
gb

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,Northern Ireland,GB-NIR,14339913B6930312138089,GBR,ADM1,"MULTIPOLYGON (((-5.97653 55.05660, -5.96920 55..."
1,England,GB-ENG,14339913B95766344400054,GBR,ADM1,"MULTIPOLYGON (((-6.32837 49.88872, -6.34348 49..."
2,Scotland,GB-SCT,14339913B23556801435424,GBR,ADM1,"MULTIPOLYGON (((-0.73807 60.42085, -0.76062 60..."
3,Wales,GB-WLS,14339913B89763821047858,GBR,ADM1,"MULTIPOLYGON (((-3.10135 53.26694, -3.00024 53..."


In [4]:
ew = gb[gb.shapeName.isin(["Wales", "England"])]

In [14]:
ew

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
1,England,GB-ENG,14339913B95766344400054,GBR,ADM1,"MULTIPOLYGON (((-6.32837 49.88872, -6.34348 49..."
3,Wales,GB-WLS,14339913B89763821047858,GBR,ADM1,"MULTIPOLYGON (((-3.10135 53.26694, -3.00024 53..."


In [15]:
%%time
grid = h3fy(ew, resolution=9, buffer=False)

CPU times: user 38.5 s, sys: 344 ms, total: 38.9 s
Wall time: 38.7 s


In [16]:
grid["hilbert"] = grid.hilbert_distance()

In [17]:
grid = grid.sort_values("hilbert").drop(columns="hilbert").to_crs(27700)
grid.to_parquet(f"{data_folder}/h3/empty_grid.parquet")

# Interpolate

## Air pollution

In [18]:
pm10_21 = (
    pd.read_csv(
        "https://uk-air.defra.gov.uk/datastore/pcm/mappm102021g.csv",
        header=5,
        na_values=["MISSING"],
    )
    .set_index(["x", "y"])
    .drop(columns="gridcode")
    .to_xarray()
)
pm25_21 = (
    pd.read_csv(
        "https://uk-air.defra.gov.uk/datastore/pcm/mappm252021g.csv",
        header=5,
        na_values=["MISSING"],
    )
    .set_index(["x", "y"])
    .drop(columns="gridcode")
    .to_xarray()
)
no2_21 = (
    pd.read_csv(
        "https://uk-air.defra.gov.uk/datastore/pcm/mapno22021.csv",
        header=5,
        na_values=["MISSING"],
    )
    .set_index(["x", "y"])
    .drop(columns="gridcode")
    .to_xarray()
)
so2_21 = (
    pd.read_csv(
        "https://uk-air.defra.gov.uk/datastore/pcm/mapso22021.csv",
        header=5,
        na_values=["MISSING"],
    )
    .set_index(["x", "y"])
    .drop(columns="gridcode")
    .to_xarray()
)
pollutants_2021 = xr.merge([pm10_21, pm25_21, no2_21, so2_21])
aqi = (
    pollutants_2021.pm252021g
    + pollutants_2021.pm102021g / 2
    + pollutants_2021.no22021 / 4
    + pollutants_2021.so22021 / 10
)
pollutants_2021 = pollutants_2021.assign(aqi=aqi)

In [19]:
bds = ew.to_crs(27700).total_bounds
pollutants_aoi = pollutants_2021.sel(x=slice(bds[0], bds[2]), y=slice(bds[1], bds[3]))
pollutants_aoi_df = pollutants_aoi.to_dataframe().reset_index()
pollutants_aoi_df = gpd.GeoDataFrame(
    pollutants_aoi_df,
    geometry=gpd.points_from_xy(
        pollutants_aoi_df.x, pollutants_aoi_df.y, crs=27700
    ).buffer(500, cap_style=3),
)

In [20]:
%%time
interp = tobler.area_weighted.area_interpolate(
    pollutants_aoi_df, grid, intensive_variables=["aqi"]
)
grid["air_quality_index"] = interp.aqi.values

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: aqi, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")


CPU times: user 48.7 s, sys: 297 ms, total: 49 s
Wall time: 49 s


In [21]:
grid[["air_quality_index"]].to_parquet(f"{data_folder}/h3/air_quality.parquet")

## House price & population

Both come from same geoms

In [22]:
house_prices = gpd.read_parquet(
    f"{data_folder}/processed/house_prices/price_per_sqm_england.parquet"
)

In [23]:
pop = gpd.read_parquet(f"{data_folder}/processed/gb_population_estimates.pq")

In [24]:
pop_hp = house_prices.merge(pop[["code", "population"]], on="code")

In [25]:
%%time
interp_oa = tobler.area_weighted.area_interpolate(
    pop_hp,
    grid,
    intensive_variables=["priceper"],
    extensive_variables=["population"],
)
grid["house_price_index"] = interp_oa.priceper.values
grid["population"] = interp_oa.population.values

CPU times: user 1min 1s, sys: 470 ms, total: 1min 2s
Wall time: 1min 2s


In [26]:
grid[["house_price_index"]].to_parquet(f"{data_folder}/h3/house_price.parquet")

In [27]:
grid[["population"]].to_parquet(f"{data_folder}/h3/population.parquet")

## Workplace population

In [28]:
wp = gpd.read_parquet(
    f"{data_folder}/raw/workplace_population/workplace_by_industry_gb.pq"
)

In [29]:
%%time
wp_interpolated = tobler.area_weighted.area_interpolate(
    wp,
    grid,
    extensive_variables=[
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ],
)

grid[
    [
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ]
] = wp_interpolated.drop(columns="geometry").values

CPU times: user 58.3 s, sys: 243 ms, total: 58.5 s
Wall time: 58.5 s


In [30]:
grid[[
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ]].to_parquet(f"{data_folder}/h3/wp_population.parquet")

## Corine

In [31]:
corine = gpd.read_parquet(f"{data_folder}/raw/land_cover/corine_gb.pq")

In [32]:
corine_names = {
    "Code_18_124": "Land cover [Airports]",
    "Code_18_211": "Land cover [Non-irrigated arable land]",
    "Code_18_121": "Land cover [Industrial or commercial units]",
    "Code_18_421": "Land cover [Salt marshes]",
    "Code_18_522": "Land cover [Estuaries]",
    "Code_18_142": "Land cover [Sport and leisure facilities]",
    "Code_18_141": "Land cover [Green urban areas]",
    "Code_18_112": "Land cover [Discontinuous urban fabric]",
    "Code_18_231": "Land cover [Pastures]",
    "Code_18_311": "Land cover [Broad-leaved forest]",
    "Code_18_131": "Land cover [Mineral extraction sites]",
    "Code_18_123": "Land cover [Port areas]",
    "Code_18_122": "Land cover [Road and rail networks and associated land]",
    "Code_18_512": "Land cover [Water bodies]",
    "Code_18_243": "Land cover [Land principally occupied by agriculture, with significant areas of natural vegetation]",
    "Code_18_313": "Land cover [Mixed forest]",
    "Code_18_412": "Land cover [Peat bogs]",
    "Code_18_321": "Land cover [Natural grasslands]",
    "Code_18_322": "Land cover [Moors and heathland]",
    "Code_18_324": "Land cover [Transitional woodland-shrub]",
    "Code_18_111": "Land cover [Continuous urban fabric]",
    "Code_18_423": "Land cover [Intertidal flats]",
    "Code_18_523": "Land cover [Sea and ocean]",
    "Code_18_312": "Land cover [Coniferous forest]",
    "Code_18_133": "Land cover [Construction sites]",
    "Code_18_333": "Land cover [Sparsely vegetated areas]",
    "Code_18_332": "Land cover [Bare rocks]",
    "Code_18_411": "Land cover [Inland marshes]",
    "Code_18_132": "Land cover [Dump sites]",
    "Code_18_222": "Land cover [Fruit trees and berry plantations]",
    "Code_18_242": "Land cover [Complex cultivation patterns]",
    "Code_18_331": "Land cover [Beaches, dunes, sands]",
    "Code_18_511": "Land cover [Water courses]",
    "Code_18_334": "Land cover [Burnt areas]",
    "Code_18_244": "Land cover [Agro-forestry areas]",
    "Code_18_521": "Land cover [Coastal lagoons]",
}

In [33]:
corine_aoi = corine.iloc[corine.sindex.query(ew.to_crs(27700).unary_union)]

In [34]:
%%time
corine_interpolated = tobler.area_weighted.area_interpolate(
    corine_aoi, grid, categorical_variables=["Code_18"]
)
corine_interpolated.columns = corine_interpolated.columns.map(corine_names)
interesting = [
    "Land cover [Discontinuous urban fabric]",
    "Land cover [Continuous urban fabric]",
    "Land cover [Non-irrigated arable land]",
    "Land cover [Industrial or commercial units]",
    "Land cover [Green urban areas]",
    "Land cover [Pastures]",
    "Land cover [Sport and leisure facilities]",
]
grid[interesting] = corine_interpolated[interesting].values

CPU times: user 1h 26min 32s, sys: 891 ms, total: 1h 26min 33s
Wall time: 1h 26min 33s


In [35]:
grid[interesting].to_parquet(f"{data_folder}/h3/corine.parquet")

## Morphometrics

In [36]:
data = dask_geopandas.read_parquet("../../../urbangrammar_samba/spatial_signatures/morphometrics/cells")

In [42]:
data.calculate_spatial_partitions()

In [37]:
chars = data.columns.drop(
    [
        "hindex",
        "tessellation",
        "buildings",
        "nodeID",
        "edgeID_keys",
        "edgeID_values",
        "edgeID_primary",
        "sdbPer",
        "ssbElo",
        "stcOri",
        "sdcLAL",
        "mdcAre",
        "ltcAre",
        "ltcWRE",
        "mtdMDi",
        "lcdMes",
        "lddNDe",
        "sddAre",
        "mdsAre",
        "ldsAre",
        "lisCel",
        "ldePer",
        "lseCWA",
    ]
)

In [43]:
interpolated = []
for i, chunk in enumerate(np.array_split(grid, 100)):
    s = time()
    mx, my, xx, xy = chunk.total_bounds
    affected_cells = data.cx[mx:xx, my:xy].compute()
    interpolated.append(tobler.area_weighted.area_interpolate(affected_cells, chunk, intensive_variables=chars.tolist()))
    del affected_cells
    gc.collect()
    print(f"Chunk {i} done in {time() - s:.2f} seconds.")

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoData

Chunk 0 done in 54.12 seconds.


/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbAre, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbCoA, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCCo, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCor, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbSqu, replacing with

Chunk 1 done in 22.84 seconds.


/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbAre, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbCoA, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCCo, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCor, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbSqu, replacing with

...

Chunk 97 done in 237.55 seconds.


/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbAre, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbCoA, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCCo, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCor, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbSqu, replacing with

Chunk 98 done in 148.30 seconds.


/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbAre, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: sdbCoA, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCCo, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbCor, replacing with 0
  warn(f"nan values in variable: {column}, replacing with 0")
/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:53: UserWarning: nan values in variable: ssbSqu, replacing with

Chunk 99 done in 98.55 seconds.


In [44]:
all_of_it = pd.concat(interpolated)

In [45]:
grid[chars] = all_of_it[chars].values

In [46]:
grid[chars].drop(columns=["ssbCCo", "ssbSqu", "ssbERI", "stbOri"]).to_parquet(f"{data_folder}/h3/morphometrics.parquet")

In [48]:
grid.drop(columns=["ssbCCo", "ssbSqu", "ssbERI", "stbOri"]).to_parquet(f"{data_folder}/h3/grid_complete.parquet")

## Signature type

In [3]:
grid = gpd.read_parquet(f"{data_folder}/h3/grid_complete.parquet")

In [4]:
signatures = gpd.read_file("../../../urbangrammar_samba/spatial_signatures/signatures/spatial_signatures_GB_simplified.gpkg", engine="pyogrio")

In [8]:
%%time
grid_signatures = grid.set_geometry(grid.centroid)[["geometry"]].sjoin(signatures, predicate="within", how="left")

CPU times: user 5.01 s, sys: 221 ms, total: 5.23 s
Wall time: 5.23 s


In [20]:
grid_signatures = grid_signatures.reset_index().drop_duplicates(subset="hex_id").set_index("hex_id")

In [21]:
grid["signature_type"] = grid_signatures["type"]

In [23]:
grid[["signature_type"]].to_parquet(f"{data_folder}/h3/signatures.parquet")

## Wales not present in house price

We don't have all of the data for Wales, let's filter it out.

In [56]:
grid_en = grid.iloc[grid.sindex.query(gb[gb.shapeName.isin(["England"])].to_crs(27700).geometry.item(), predicate="intersects")]

In [58]:
grid_en.drop(columns=["ssbCCo", "ssbSqu", "ssbERI", "stbOri"]).to_parquet(f"{data_folder}/h3/grid_complete.parquet")

In [59]:
grid_en[interesting].to_parquet(f"{data_folder}/h3/corine.parquet")

In [60]:
grid_en[[
        "A, B, D, E. Agriculture, energy and water",
        "C. Manufacturing",
        "F. Construction",
        "G, I. Distribution, hotels and restaurants",
        "H, J. Transport and communication",
        "K, L, M, N. Financial, real estate, professional and administrative activities",
        "O,P,Q. Public administration, education and health",
        "R, S, T, U. Other",
    ]].to_parquet(f"{data_folder}/h3/wp_population.parquet")

In [61]:
grid_en[["population"]].to_parquet(f"{data_folder}/h3/population.parquet")

In [62]:
grid_en[["house_price_index"]].to_parquet(f"{data_folder}/h3/house_price.parquet")

In [63]:
grid_en[["air_quality_index"]].to_parquet(f"{data_folder}/h3/air_quality.parquet")

In [64]:
grid_en[chars].drop(columns=["ssbCCo", "ssbSqu", "ssbERI", "stbOri"]).to_parquet(f"{data_folder}/h3/morphometrics.parquet")

In [65]:
grid[["geometry"]].to_parquet(f"{data_folder}/h3/empty_grid.parquet")

In [20]:
grid = h3fy(ew, resolution=3, buffer=True)

/home/martin/mambaforge/envs/stable/lib/python3.12/site-packages/tobler/util/util.py:113: UserWarning: The source geodataframe is stored in a geographic CRS. Falling back to estimated UTM zone to generate desired buffer. If this produces unexpected results, reproject the input data prior to using this function
  warn(


In [21]:
grid.explore()

In [17]:
grid.shape

(16, 1)